# SQL analysis notebook 
## purpose : 
    - Analyse the data stored on the *swapi.db* database
    - Generate tables to be used on the visualization notebook and the final_report
    - Conclude insights and find patterns 
## Data :
    - Data used in this notebook is found on the swapi.db database, all tables in the database are pulled from the SWAPI api and generated in the api_ingestion notebook 
## Results :
    - By running this notebook, tables will be generated in this folder :  ../Output/tables/

In [1]:
# Importing tools needed 
import sqlite3
import pandas as pd

In [2]:
# connect to the Database (swapi.db)
conn = sqlite3.connect("../Database/swapi.db")

### Loading the tables 

In [63]:
# Creating a function to speed up the process ;
def connect(table) :
    return  pd.read_sql(f"select * from {table}", conn)

# Using the "connect" function to load the tables ;
people = connect("people")
starships = connect("starships")
species = connect("species")
planets = connect("planets")
films = connect("films")
vehicles = connect("vehicles")
films_characters = connect("films_characters")
films_planets = connect("films_planets")
films_species = connect("films_species")
people_starships = connect("people_starships")
people_vehicles = connect("people_vehicles")

In [4]:
# Creating a simple function to speed up the process of saving the newly created tables
def save(new_table, name) :
    new_table.to_csv(f"../Output/tables/{name}.csv", index = False)

### Tallest people

In [5]:
tallest_ppl = (people[people["height"] != "unknown"]).copy()
tallest_ppl["height"] = tallest_ppl["height"].astype(int)
tallest_ppl = tallest_ppl.sort_values(by="height",ascending=False)[["name","height"]]
save(tallest_ppl, "tallest_ppl")
tallest_ppl.head(5)

,name,height
55,Yarael Poof,264
78,Tarfful,234
70,Lama Su,229
12,Chewbacca,228
35,Roos Tarpals,224


> As we can see in the output table, The tallest person is **Yarael Poof** with a height of **264**

### People with the highest mass

In [6]:
heaviest_ppl = (people[people["mass"] != "unknown"]).copy()
heaviest_ppl = heaviest_ppl.sort_values(by="mass",ascending=True)[["name","mass"]]
save(heaviest_ppl, "heaviest_ppl")
heaviest_ppl.head(5)

,name,mass
15,Jabba Desilijic Tiure,"1,358"
69,Dexter Jettster,102
17,Jek Tono Porkins,110
12,Chewbacca,112
22,Bossk,113


>  As we can see in the output table, The person with the highest mass is **Jabba Desilijic Tiure** with a mass of **1,358**

### Oldest people

In [7]:
oldest_ppl = people[people["birth_year"] != "unknown"].copy()
oldest_ppl.loc[:, "birth_year_num"] = (oldest_ppl["birth_year"]
    .str.extract(r"(\d+)")
    .astype(int))
oldest_ppl = (oldest_ppl
    .sort_values(by="birth_year_num",ascending=False)[["name", "birth_year"]])
save(oldest_ppl, "oldest_ppl")
oldest_ppl.head(3)

,name,birth_year
18,Yoda,896BBY
15,Jabba Desilijic Tiure,600BBY
12,Chewbacca,200BBY


> As we can see in the output table, the oldest person is **Yoda** with an age of *896 years before the battle of yavin (896BBY)*

### Oldest films

In [8]:
films["release_date"] = pd.to_datetime(films["release_date"])
oldest_films = (films
    .sort_values("release_date", ascending=True))[["title", "release_date"]]
save(oldest_films, "oldest_films")
oldest_films.head(5)

,title,release_date
0,A New Hope,1977-05-25
1,The Empire Strikes Back,1980-05-17
2,Return of the Jedi,1983-05-25
3,The Phantom Menace,1999-05-19
4,Attack of the Clones,2002-05-16


> As we can see in the output table, The oldest movie is **"A New Hope"** with a release date of 1977-05-25

### Film episode_id ascending

In [9]:
episode_asc = films.sort_values("episode_id", ascending=True)[['title','episode_id']]
save(episode_asc, "episode_asc")
episode_asc

,title,episode_id
3,The Phantom Menace,1
4,Attack of the Clones,2
5,Revenge of the Sith,3
0,A New Hope,4
1,The Empire Strikes Back,5
2,Return of the Jedi,6


> As we can see in the output table, The first movie episode is **"The Phantom Menace"**

### Movies with the most characters

In [10]:
films_characters_num = films_characters.copy()
films_characters_num["characters"] = (films_characters_num["characters"]
    .str.split('/').str[-2])
films_by_character_count = (films_characters_num
    .value_counts("title")
    .reset_index(name="character_count")
    .sort_values(by="character_count",ascending=False))
save(films_by_character_count,"films_by_character_count")
films_by_character_count.head(3)

,title,character_count
0,Attack of the Clones,40
1,Revenge of the Sith,34
2,The Phantom Menace,34


> As we can see in the output table, The movie with the most characters is **"Attack of the Clones"** with 40 characters

### Most frequent characters

In [11]:
people["id"] = people["url"].str.split('/').str[-2]
most_frequent_characters = (films_characters_num["characters"]
    .value_counts()
    .reset_index(name="movie_count")).copy()
most_frequent_characters = (most_frequent_characters
    .rename(columns={"characters" : "id"}))
most_frequent_characters_people = (most_frequent_characters
    .merge(people,on="id",how="left"))
most_frequent_characters["movie_count"] = most_frequent_characters["movie_count"].astype(int)
most_frequent_characters = (most_frequent_characters_people
    .sort_values("movie_count", ascending=False)[["name", "movie_count"]]).copy()
save(most_frequent_characters,"most_frequent_characters")
most_frequent_characters.head(5)

,name,movie_count
0,C-3PO,6
1,R2-D2,6
2,Obi-Wan Kenobi,6
3,Yoda,5
4,Palpatine,5


> As we can see in the output table, The most frequent characters are : **C-3PO, R2-D2, Obi-Wan Kenobi**, Who all appeared in 6 films

### Tallest species (by average height)

In [12]:
tallest_sp = species[
    (species["average_height"] != "unknown") &
    (species["average_height"] != "n/a")
    ].copy()
tallest_sp["average_height"] = tallest_sp["average_height"].astype(int)
tallest_sp = (tallest_sp
    .sort_values('average_height',ascending=False)
    )[["name", 'average_height']]
save(tallest_sp, "tallest_sp")
tallest_sp.head(5)

,name,average_height
4,Hutt,300
24,Quermian,240
31,Kaminoan,220
2,Wookie,210
18,Toong,200


> As we can see in the output table, The tallest species by average height is called the **"Hutt"** with an average height of **300**

### Planets with the highest population 

In [13]:
planets_population = (planets[planets["population"] != "unknown"]).copy()
planets_population["population"]= planets_population["population"].astype(int) 
planets_population = (planets_population
                      .sort_values("population",ascending=False)
                     [["name", "population"]])
save(planets_population, "planets_population")
planets_population.head(5)

,name,population
8,Coruscant,1000000000000
55,Skako,500000000000
10,Geonosis,100000000000
30,Mon Cala,27000000000
20,Eriadu,22000000000


> As we can see in the output table, The planet with the highest population is **"Coruscant"**

### Planets with the highest surface_water

In [14]:
planets_water = (planets[planets["surface_water"] != "unknown"]).copy()
planets_water["surface_water"]= pd.to_numeric(planets_water["surface_water"])
planets_water = (planets_water
                      .sort_values("surface_water",ascending=False)
                     [["name", "surface_water"]])
save(planets_water, "planets_water")
planets_water.head(5)

,name,surface_water
9,Kamino,100.0
3,Hoth,100.0
30,Mon Cala,100.0
54,Ojom,100.0
25,Bestine IV,98.0


> As we can see in the output table, The planets with the highest water_surface are : **Kamino, Hoth, Mon Cala and Ojom** with a water surface of *100.0*

### Planets with the most films 

In [20]:
planets["planet_id"] = planets["url"].astype(str).str.split("/").str[-2] 
top_planets_by_films = (films_planets["planets"]
                       .str.split("/")
                       .str[-2]
                       .value_counts()
                       .reset_index(name="film_count")
                       .rename(columns={"planets" : "planet_id"}) 
                      ).copy()
planets["planet_id"] = planets["planet_id"].astype(str)
top_planets_by_films_planets = (planets
    .merge(top_planets_by_films, on="planet_id", how="left")
                               )
top_planets_by_films = (top_planets_by_films_planets[["name", "film_count"]]
    .sort_values("film_count", ascending=False)
                       ).copy()
top_planets_by_films = (
    top_planets_by_films[top_planets_by_films["film_count"].notna()] 
)
save(top_planets_by_films, "top_planets_by_films")
top_planets_by_films.head(5)

,name,film_count
0,Tatooine,5.0
7,Naboo,4.0
8,Coruscant,4.0
4,Dagobah,3.0
1,Alderaan,2.0


> As we can see in the output table, The planet with the most films is **Tatooine** with *5* films

### Most frequent **species** in films

In [31]:
species["species_id"] = species["url"].astype(str).str.split("/").str[-2] 
species["species_id"] = species["species_id"].astype(str)
frequent_species = (films_species["species"]
                       .str.split("/")
                       .str[-2]
                       .value_counts()
                       .reset_index(name="film_count")
                       .rename(columns={"species" : "species_id"}) 
                      ).copy()
frequent_species_species = (
    species.merge(frequent_species, on="species_id", how="left")
)
frequent_species = (frequent_species_species[["name", "film_count"]]
                    .sort_values("film_count",ascending=False)
                   ).copy()
save(frequent_species, "frequent_species")
frequent_species.head(3)

,name,film_count
0,Human,6
1,Droid,6
5,Yoda's species,5


> As we can see in the output table, The most frequent species is **Humans** and **Droid** with 6 films 

### People with the most starships

In [59]:
# Creating a function to speed up the process of analysing people related tables
def people_analyse(table, friend_table, new_table_name) :
    people["people_id"] = people["url"].astype(str).str.split("/").str[-2] 
    people["people_id"] = people["people_id"].astype(str)
    friend_table = friend_table[(friend_table[f"{table}"] != "None") & (friend_table[f"{table}"].notna())]
    friend_table.loc[:, f"{table}"] = friend_table[f"{table}"].astype(str).str.split("/").str[-2] 
    new_table = (friend_table["name"]
                       .value_counts()
                       .reset_index(name=f"{table}_count")
                      ).copy()
    new_table = new_table[new_table[f"{table}_count"].notna()]
    save(new_table, f"{new_table_name}")
    return new_table.head(5)
people_analyse("starships", people_starships, "top_people_by_starships") 

,name,starships_count
0,Obi-Wan Kenobi,5
1,Anakin Skywalker,3
2,Padmé Amidala,3
3,Han Solo,2
4,Chewbacca,2


> As we can see in the output table, The person with the most starships is **Obi-Wan Kenobi** with *5 starships*

### People with the most vehicles 

In [61]:
# Using people_analyse to speed up the process
people_analyse('vehicles', people_vehicles, "top_people_by_vehicles")

,name,vehicles_count
0,Luke Skywalker,2
1,Anakin Skywalker,2
2,Leia Organa,1
3,Obi-Wan Kenobi,1
4,Chewbacca,1


> As we can see in the output table, The person with the most vehicles is **Luke Skywalker** and **Anakin Skywalker** with *2 vehicles*